# Genome-scale reconstruction workflow

This notebook serves as a step-by-step workflow for improving or reconstructing metabolic models and showcasing the functionality of the MetMod library.

It is a part of a bachelor's thesis, where the theoretical aspects of metabolic model reconstruction are introduced and discussed.

## 0. Dependencies and setup

We recommend using Python 3.9 as this version's functionality has been tested.

The following libraries are required:
* cobra, escher, networkx, matplotlib, xlsxwriter, pandas, numpy, openpyxl

Additional:
* ModelPolisher, efmtool, Cplex
* libsbml - https://sbml.org/software/libsbml/libsbml-docs/installation/

If there are any problems with Escher visualisation, refer to https://escher.readthedocs.io/en/latest/escher-python.html#Installation, as further Jupyter extensions may be required. When using a conda environment, make sure Jupyter is installed inside it. These commands worked when testing:
* jupyter labextension install @jupyter-widgets/jupyterlab-manager
* jupyter labextension install escher

We recommend using the academic or commercial license of Cplex for optimisation; the free version will not work due to the size of metabolic models.

Using a Conda environment is encouraged; the following cell contains commands tested to create a Conda environment and enable Escher visualisation within a Jupyter notebook.

In [ ]:
# conda create --name metmod-test python=3.9 -y
# conda activate metmod-test
# conda install pip -y
# conda install jupyter
# pip install escher notebook
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install escher

Run the following cell in order to install dependencies

In [ ]:
%pip install cobra escher networkx git+https://github.com/draeger-lab/MPClient.git matplotlib xlsxwriter pandas numpy efmtool openpyxl

In [ ]:
import cobra, escher, libsbml, os
import numpy as np
import matplotlib as plt
import networkx as nx
import metmod

In [ ]:
model_name = 'ToyModel'

In [ ]:
# Some models created by automated tools will have some issues, like not having flux bounds set
# correctly, in order to suppress warnings run this cell
import logging

logging.getLogger('cobra').setLevel(logging.ERROR)

## 1. Draft reconstruction

A draft reconstruction is expected as input; we recommend using an automated software designed for this purpose, such as PathwayTools - PathoLogic or CarveMe.

### Standardising nomenclature

ModelPolisher is used in order to standardise and polish annotations of the draft reconstruction.

**NOTE** polishing bigger models (>1000 reactions) may take up to few minutes

In [ ]:
input_path = os.path.join('input', 'input.xml')

In [ ]:
polished_path = os.path.join('models', f'{model_name}_polished.xml')

In [ ]:
metmod.polish_model(input_path, polished_path, print_diff=True)

## 2. First assessment

In the section we will take a first look at the draft reconstruction. We highly encourage the use of Cytoscape for this step.

### Escher

While Escher does not provide a way to visualise the entire network, it allows for easy mapping of the model to an existing visualisation.

However, the IDs in the model must match the IDs in the map, depending on the software used for draft reconstruction **this approach may not work**.

In [ ]:
# The available maps are listed by the following command.
escher.list_available_maps()

In [ ]:
map_path = os.path.join('maps', 'e_coli_core.Core metabolism.json')

Escher provides a Builder class where all settings are configured. Here we use an existing map with our model.

In [ ]:
metmod.escher_build(polished_path, 'e_coli_core.Core metabolism', online_map=True, highlight_missing=True)

Reactions highlighted in red are missing from the model; however note that this may also be the cause of using different nomenclature.

When using Pathologic, for example, there will be nomenclature differences. Therefore, the following function tries to scrape BiGG identifiers provided by ModelPolisher and change the nomenclature of an Escher map.

In [ ]:
metmod.escher_map_sync(map_path, polished_path)

In [ ]:
renamed_map = os.path.join('maps', 'renamed_e_coli_core.Core metabolism.json')

In [ ]:
metmod.escher_build(polished_path, renamed_map, highlight_missing=True)

Hopefully some pathways are now blue, if that is not the case manual renaming is required.

### Topology analysis

NetworkX is used for graph algorithms, some wrappers are present in MetMod.

In [ ]:
model = cobra.io.read_sbml_model(polished_path)

Firstly, a function that creates a directed G graph is called.

In [ ]:
G = metmod.create_metabolic_graph(model)

In [ ]:
# Print out nodes sorted by node degree (+reversed)
metmod.print_degree_sorted(G)

In [ ]:
metmod.print_degree_distribution(G)

In [ ]:
# Single node strongly connected components are omitted
metmod.print_strongly_connected(G)

Some helper functions should make it easy to work with the graph, as naming can be unintuitive. Creation of variables instead of relying on strings is encouraged.

In [ ]:
# Inspect all node names
# metmod.print_graph_nodes_names()
# Find for names containing a string
metmod.find_graph_names_with(G, 'ADP')

In [ ]:
glu = metmod.get_id_by_name(G, 'D-Glucose')
pyr = metmod.get_id_by_name(G, 'Pyruvate')
eno = metmod.get_id_by_name(G, 'Enolase')

In [ ]:
metmod.print_neighbors(G, glu)

In [ ]:
_, named = metmod.find_paths(G, glu, pyr, [eno], 10, print_path=True)

In [ ]:
print(metmod.get_id_by_name(G, 'ADP'))

In [ ]:
# Threshold needs to be tinkered with and set based on degree distribution
# Check if any important nodes were deleted
pruned_G = metmod.prune_graph(G, 30, [pyr])

# Manually delete some nodes
pruned_G = metmod.delete_nodes_by_id(pruned_G, ['adp_c', metmod.get_id_by_name(G, 'Ecoli_core model biomass')])
 
ids, named = metmod.find_paths(pruned_G, glu, pyr, [eno], 10, print_path=True)

## 3. Iterative workflow

### 3.1 Refinement

Firstly we will create additional files in order to ease manual refinement of model.

In [ ]:
xlsx_path = os.path.join('models', f'{model_name}.xlsx')

In [ ]:
model = cobra.io.read_sbml_model(polished_path)
# Create a xlsx file from cobra model
metmod.cobra_to_xlsx(model, xlsx_path)

In [ ]:
# Sync xlsx file with SBML and json
metmod.sync_xlsx(xlsx_path, os.path.join('models', model_name))

In [ ]:
# Optionally load a cobra model from xlsx
model = metmod.xlsx_to_cobra(xlsx_path, model_name)

Additionally, for working with Escher maps and creating a core model, a function which deletes all compounds not present in a map is available. 

In [ ]:
metmod.mini_model_prune(map_path, polished_path, f'mini_{model_name}')
mini_model_path = f'mini_{model_name}.xml'

### 3.2 Cross checking

#### Flux Balance Analysis

Make sure the model has an objective function.

In [ ]:
print(model.objective)          # Shows the symbolic expression
print(model.objective.expression)  # Explicit formula (reaction * coefficient)
print(model.objective.direction)   # 'max' or 'min'

In [ ]:
# If the model contains a biomass reaction, it can be set as objective:
biomass = model.reactions.get_by_id('Biomass')
model.objective = biomass

A quick way to test a biomass function may be to use an existing one from a different model, metmod.filter_reaction deletes all metabolites in the reaction that are not found in the model.

In [ ]:
ecoli_biomass_core = """
1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c +
4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c 
→ 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
"""
# metmod.filter_reaction(ecoli_biomass_core, model, arrow_symbol="→")
# filter_reaction_bigg additionally renames metabolites in the formula to match metabolites in the model, uses ModelPoliher BiGG annotation
reaction = metmod.filter_reaction_bigg(ecoli_biomass_core, polished_path, arrow_symbol="→")
print(reaction)

In [ ]:
# With formula string, we can add the objective function
metmod.add_objective_function(reaction, model)

In [ ]:
solution = model.optimize()
solution

Run and visualise FBA in Escher

In [ ]:
metmod.escher_build(polished_path, renamed_map, reaction_data=model.optimize().fluxes)

Knockout a gene.

In [ ]:
print(model.genes)

In [ ]:
model.genes.IS481_03595.knock_out()

In [ ]:
metmod.escher_build(polished_path, renamed_map, reaction_data=model.optimize().fluxes)

#### Elementary Flux Mode analysis
* Note that efmtool requires Java 8; one way of switching Java versions is as follows: export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)

*Since enumerating EFMs is complex, we will not use it on the draft model.*

In [ ]:
model = cobra.io.read_sbml_model(polished_path)

If the cell is taking a long time to run, try turning the logger on to see if there is an issue.

In [ ]:
efm_matrix = metmod.calculate_efm_matrix(model, log=True)

In [ ]:
smallest_efm_idx = metmod.find_smallest_efm(efm_matrix)
smallest_efm = efm_matrix[:, smallest_efm_idx]

In [ ]:
all_efms = metmod.combine_all_efms(efm_matrix)

In [ ]:
print(metmod.find_reaction_not_in_efms(efm_matrix, model))

We can again visualise data with Escher; efm_for_escher returns reaction data. Alternatively, reaction data can be saved to a file.

In [ ]:
metmod.escher_build(polished_path, "renamed_e_coli_core.Core metabolism.json", reaction_data=metmod.efm_for_escher(model, all_efms))

Check how many EFMs contain a specific reaction. Reaction ID is used.

In [ ]:
efms_with_biomass = metmod.filter_efms(efm_matrix, ["BIOMASS_Ecoli_core_w_GAM"], model)
len(efms_with_biomass)

#### Gene expression data

In [ ]:
gene_expression_data = metmod.filter_gene_csv(os.path.join('expression_data', 'RNASeq2_RAW_CountTable.csv'), 1, 5, os.path.join('expression_data', 'filtered_draft_genes.csv'), delim=';')

In [ ]:
metmod.sync_map_genes(polished_path, renamed_map, 'draft_gene_map.json')

Mapping transcriptomics data to a map.

If gene data is not showing properly, manually import it in Escher. *Data -> Load gene data*

In [ ]:
metmod.escher_build(polished_path, os.path.join('maps', 'draft_gene_map.json'), gene_data=gene_expression_data,)